In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('../input/spam-text-message-classification/SPAM text message 20170820 - Data.csv')

In [ ]:
df['Category'].value_counts()

In [ ]:
spam = df[df['Message'].str.contains('win' and 'free')]
spam['Category'].value_counts()

In [ ]:
ham_message_length = []
spam_message_length = []
for i in df.values:
    if(i[0] == 'ham'):
        ham_message_length.append(len([i]))
    else:
        spam_message_length.append(len([i]))
ham_message_length

In [ ]:
import pandas as pd
from gensim.models.word2vec import Word2Vec
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.layers import Dense, Dropout, Conv1D, MaxPool1D, GlobalMaxPool1D, Embedding, Activation
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import PorterStemmer
from sklearn import preprocessing

In [ ]:
def preprocess_text(sen):
    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]',' ',sen)
    # Single character removal
    sentence = re.sub(r'\s+[a-zA-Z]\s+', ' ', sentence)
    # Removing multiple spaces
    sentence = re.sub(r'\s+',' ', sentence)
    
    stops = stopwords.words('english')
    
    porter = PorterStemmer()
    for word in sentence.split():
        if word in stops:
            sentence = sentence.replace(word, '')
        sentence = sentence.replace(word, porter.stem(word))
    return sentence.lower()
df['Message'] = df['Message'].apply(preprocess_text)

In [ ]:
mes =[]
for i in df["Message"]:
    mes.append(i.split())
print(mes[:2])

In [ ]:
word2vec_model = Word2Vec(mes, size = 500, window = 3, min_count=1, workers=16)
print(word2vec_model)

In [ ]:
token = Tokenizer(7229)
token.fit_on_texts(df['Message'])
text = token.texts_to_sequences(df['Message'])
text = pad_sequences(text, 75)

In [ ]:
le = preprocessing.LabelEncoder()
y = le.fit_transform(df['Category'])
y = to_categorical(y)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(np.array(text), y, test_size = 0.2, stratify = y)

In [ ]:
import tensorflow as tf
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units = 110, activation = 'relu'))
ann.add(tf.keras.layers.Dense(units =110, activation ='relu'))
ann.add(tf.keras.layers.Dense(units = 2, activation ='sigmoid'))
ann.compile(optimizer = 'adam', loss='binary_crossentropy', metrics=['accuracy'])
ann.fit(x_train, y_train, batch_size=32, epochs=100)